# Zip folder unpacking script
This script will do the following operations in order:
Scan a folder
create new calculation folder
Loop the following actions for each homer script


    Unseal homer template file (.homer -> .zip)
    
    
    Unpack zip file (.zip -> normal directory)
    
    
    Open 'problem' file
    
    
    ####### NOT FOR THIS SCRIPT modify 'problem' file
    
    
    Repack zip file (normal directory -> .zip)
    
    
    Reseal homer file (.zip -> .homer)
    
    
    store updated input file in calculation folder
    
    
    end 

In [44]:
# finding all noteworthy files
import os
import numpy as np 
import zipfile
import shutil
import zlib
import pathlib 
import subprocess
from lxml import etree, objectify
folder = 'c:/repo/homer/ClimateStudies/XPathTesting/HOMERAutomation'
files = [f for f in os.listdir(folder) if f.endswith('.homer')]
n_files = np.size(files)
display(files)

['Case1BurlingtonVTDfa.homer',
 'Case1ChicagoDfa.homer',
 'Case1ColumbusOHDfa.homer',
 'Case1DesMoinesIADfa.homer',
 'Case1DetroitMIDfa.homer',
 'Case1LincolnNEDfa.homer',
 'Case1SouthBendINDfa.homer']

In [45]:
# Unsealing HOMER template files (.homer -> .zip)
for i in range(len(files)):
    file = files[i]
    base = os.path.splitext(file)[0]
    os.rename(file, base + '.zip')

In [46]:
# Unpack zip file (.zip -> normal directory)
files = [f for f in os.listdir(folder) if f.endswith('.zip')]
n_files = np.size(files)
for i in range(len(files)):
    file = files[i]
    base = os.path.splitext(file)[0]
    with zipfile.ZipFile(file, 'r') as zip_ref:
        zip_ref.extractall(base + 'TEMP')
    os.remove(file)
# Opening each individiual 'problem' file after extraction
    

In [47]:
# (Within normal directory) Edit the .xml files for each HOMER problem.
files = [f for f in os.listdir(folder) if f.endswith('TEMP')]
display(files)
# paths to change as key, new value as value
new_data = {'/HomerProblem/FuelResources/Fuel/Name': 'CCM FUEL',
            '/HomerProblem/Generators/Generator/CostMatrix/Cost/Initial': '8000000',
            '/HomerProblem/Generators/Generator/CostMatrix/Cost/CapitalCostDetail/CostCalculatorItem/Quantity': '1000' ,
            '/HomerProblem/Generators/Generator/CostMatrix/Cost/CapitalCostDetail/CostCalculatorItem/UnitCost': '8000' ,
            '/HomerProblem/Generators/Generator/CostMatrix/Cost/Replacement': '3550000',
            '/HomerProblem/Generators/Generator/CostMatrix/Cost/ReplacementCostDetail/CostCalculatorItem/Quantity': '1000' ,
            '/HomerProblem/Generators/Generator/CostMatrix/Cost/ReplacementCostDetail/CostCalculatorItem/UnitCost': '3550000' ,
            '/HomerProblem/Generators/Generator/CostMatrix/Cost/Operating': '14.60',
            '/HomerProblem/SimulationParameters/Economics/ProjectLifetimeSensitivity': '60'}
   
for i in range(len(files)):
    file = files[i]
    problems = etree.parse(file + '/problem')
    for path, value in new_data.items():
        if isinstance(value,str) == True:
            print(path, ' => ', problems.xpath(path)[0].text)
            problems.xpath(path)[0].text = value
            print(path, ' => ', problems.xpath(path)[0].text)
    print(problems)
    #with open(file + '/problem','w') as f:
    #    f.write(problems)
        
        
        
    #display(problem)
       # problem.xpath(j)[0].text = value[j]

['Case1BurlingtonVTDfaTEMP',
 'Case1ChicagoDfaTEMP',
 'Case1ColumbusOHDfaTEMP',
 'Case1DesMoinesIADfaTEMP',
 'Case1DetroitMIDfaTEMP',
 'Case1LincolnNEDfaTEMP',
 'Case1SouthBendINDfaTEMP']

/HomerProblem/FuelResources/Fuel/Name  =>  Diesel
/HomerProblem/FuelResources/Fuel/Name  =>  CCM FUEL
/HomerProblem/Generators/Generator/CostMatrix/Cost/Initial  =>  1000000000
/HomerProblem/Generators/Generator/CostMatrix/Cost/Initial  =>  8000000
/HomerProblem/Generators/Generator/CostMatrix/Cost/CapitalCostDetail/CostCalculatorItem/Quantity  =>  1
/HomerProblem/Generators/Generator/CostMatrix/Cost/CapitalCostDetail/CostCalculatorItem/Quantity  =>  1000
/HomerProblem/Generators/Generator/CostMatrix/Cost/CapitalCostDetail/CostCalculatorItem/UnitCost  =>  500
/HomerProblem/Generators/Generator/CostMatrix/Cost/CapitalCostDetail/CostCalculatorItem/UnitCost  =>  8000
/HomerProblem/Generators/Generator/CostMatrix/Cost/Replacement  =>  1000000000
/HomerProblem/Generators/Generator/CostMatrix/Cost/Replacement  =>  3550000
/HomerProblem/Generators/Generator/CostMatrix/Cost/ReplacementCostDetail/CostCalculatorItem/Quantity  =>  1
/HomerProblem/Generators/Generator/CostMatrix/Cost/ReplacementCo

/HomerProblem/FuelResources/Fuel/Name  =>  CCM FUEL
/HomerProblem/Generators/Generator/CostMatrix/Cost/Initial  =>  1000000000
/HomerProblem/Generators/Generator/CostMatrix/Cost/Initial  =>  8000000
/HomerProblem/Generators/Generator/CostMatrix/Cost/CapitalCostDetail/CostCalculatorItem/Quantity  =>  1
/HomerProblem/Generators/Generator/CostMatrix/Cost/CapitalCostDetail/CostCalculatorItem/Quantity  =>  1000
/HomerProblem/Generators/Generator/CostMatrix/Cost/CapitalCostDetail/CostCalculatorItem/UnitCost  =>  500
/HomerProblem/Generators/Generator/CostMatrix/Cost/CapitalCostDetail/CostCalculatorItem/UnitCost  =>  8000
/HomerProblem/Generators/Generator/CostMatrix/Cost/Replacement  =>  1000000000
/HomerProblem/Generators/Generator/CostMatrix/Cost/Replacement  =>  3550000
/HomerProblem/Generators/Generator/CostMatrix/Cost/ReplacementCostDetail/CostCalculatorItem/Quantity  =>  1
/HomerProblem/Generators/Generator/CostMatrix/Cost/ReplacementCostDetail/CostCalculatorItem/Quantity  =>  1000
/Ho

In [49]:
#Repack zip file (normal directory -> .zip)
## THIS STEP CURRENTLY BREAKS THE HOMER FILE
## Try using zipcode compression 
files = [f for f in os.listdir(folder) if f.endswith('TEMP')]
#directory = pathlib.Path('zipfoldertesting/') ## NEEDS TO BE In FOR LOOP, DIRECTORY -> zip file
for i in range(len(files)):
    file = files[i]
    directory = pathlib.Path(file)
    basefile = file.replace('TEMP','')
    subprocess.run(["powershell.exe", "Compress-Archive " + basefile])
    #shutil.rmtree(file) #deletes old temp folders
    

In [48]:
# Reseal homer file (.zip -> .homer)
files = [f for f in os.listdir(folder) if f.endswith('.zip')]
n_files = np.size(files)
type(files)
for i in range(len(files)):
    file = files[i]
    base = os.path.splitext(file)[0]
    os.rename(file, base + '.homer')